In [ ]:
import os
import torch
from tortoise.api import TextToSpeech
from scipy.io.wavfile import write

# Tortoise-TTS 모델 로드
tts = TextToSpeech()

# 텍스트 파일 읽기
input_file = "input_level1.txt"
output_folder = "tortoise_output"
os.makedirs(output_folder, exist_ok=True)

# 음성 샘플 경로가 있는 경우 화자 스타일 지정
# voice_samples = ["path/to/sample1.wav", "path/to/sample2.wav"]  # 화자 음성 샘플이 있을 경우 설정

with open(input_file, "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        text = line.strip()
        if not text:
            continue
        
        # 음성 생성 (voice_samples가 없으면 기본 화자로 생성)
        audio = tts.tts(text=text)  # voice_samples 매개변수를 추가해 사용할 수도 있습니다.

        # 텐서를 NumPy 배열로 변환 후 저장
        audio_numpy = audio.cpu().numpy()  # GPU에서 작업할 경우 CPU로 이동
        output_path = os.path.join(output_folder, f"speech_{idx+1}.wav")
        write(output_path, 22050, audio_numpy)  # 22050은 표준 샘플링 레이트입니다.
        
        print(f"'{text}' 문장이 '{output_path}'로 저장되었습니다.")


Generating autoregressive samples..


100%|██████████| 512/512 [1:19:00<00:00,  9.26s/it]


Computing best candidates using CLVP


100%|██████████| 512/512 [06:06<00:00,  1.40it/s]


Transforming autoregressive outputs into audio..


...

Transforming autoregressive outputs into audio..


100%|██████████| 100/100 [00:53<00:00,  1.87it/s]


'A, is for alligator.' 문장이 'tortoise_output\speech_3.wav'로 저장되었습니다.
Generating autoregressive samples..


 30%|██▉       | 152/512 [19:22<45:53,  7.65s/it]  


KeyboardInterrupt: 

In [2]:
import os
import torch
import torchaudio
from tortoise.api import TextToSpeech
from scipy.io.wavfile import write


In [12]:
# Tortoise-TTS 모델 로드
tts = TextToSpeech()


c:\Users\user\Desktop\빅데이터9기_수업자료_이수지\2024.11.04_(중간프로젝트)\tortoise-tts\tortoise\api.py:221: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.autoregressive.load_state_dict

In [16]:
# LJSpeech 음성 샘플 파일을 torch.Tensor로 로드
voice_samples = []
for sample_path in ["LJ001-0001.wav", "LJ001-0002.wav"]:
    waveform, sample_rate = torchaudio.load(sample_path)
    voice_samples.append(waveform)

In [17]:
# 변환할 텍스트
text = "This is an example that specifies the speaker style using the LJSpeech dataset."

In [19]:
# Tortoise-TTS를 사용한 음성 생성

audio = tts.tts(text=text, voice_samples=voice_samples)

Generating autoregressive samples..


  6%|▌         | 31/512 [20:47<5:22:38, 40.25s/it]


KeyboardInterrupt: 

In [ ]:
# 오디오 파일로 저장
output_path = "output_ljspeech_style.wav"
audio_numpy = audio.cpu().numpy()
write(output_path, 22050, audio_numpy)

print(f"'{text}' 문장이 '{output_path}'로 저장되었습니다.")

In [4]:
import librosa

# 오디오 파일 경로
audio_path = "LJ001-0001.wav"

# 오디오 파일 로드 및 샘플링 속도 확인
data, sample_rate = librosa.load(audio_path, sr=None)  # sr=None으로 지정하면 원본 샘플링 속도 유지
print(f"원본 오디오의 샘플링 속도는 {sample_rate} Hz입니다.")


원본 오디오의 샘플링 속도는 22050 Hz입니다.


### 영국여자 샘플 1 -> 음성추출

In [1]:
import os
import librosa
import numpy as np
from tortoise.api import TextToSpeech
from tortoise import utils
from scipy.io.wavfile import write

In [6]:
# 저장할 폴더가 없으면 생성
output_folder = 'tortoise_output1'
os.makedirs(output_folder, exist_ok=True)

# Tortoise TTS 모델 초기화
tts = TextToSpeech(use_deepspeed=True, kv_cache=True, half=True)

# 22,050Hz 샘플링 속도의 원본 오디오 파일을 로드하여 사용
voice_samples = [utils.audio.load_audio("LJ001-0001.wav", 22050)]

# Tortoise에 화자 프로필 생성
conditioning_latents = tts.get_conditioning_latents(voice_samples)

# 예시용 텍스트
text = "C is for cat."

# 텍스트를 화자 샘플의 스타일로 음성 변환
pcm_audio = tts.tts_with_preset(text, conditioning_latents=conditioning_latents, preset="ultra_fast")

# PCM 데이터를 numpy 배열로 변환 후 최종 WAV 파일로 저장
output_path = os.path.join(output_folder, 'tortoise_test_ultra_fast.wav')
write(output_path, 22050, np.array(pcm_audio, dtype=np.float32))

print(f"'{text}' 문장이 '{output_path}'로 저장되었습니다.")


Generating autoregressive samples..


100%|██████████| 16/16 [00:29<00:00,  1.83s/it]


Computing best candidates using CLVP


100%|██████████| 16/16 [00:08<00:00,  1.78it/s]


Transforming autoregressive outputs into audio..


100%|██████████| 30/30 [00:04<00:00,  6.00it/s]

'C is for cat.' 문장이 'tortoise_output1\tortoise_test_ultra_fast.wav'로 저장되었습니다.


### 영국여자 샘플 5 -> 음성추출

In [11]:
import os
import numpy as np
from tortoise.api import TextToSpeech
from tortoise import utils
from scipy.io.wavfile import write

In [12]:
# 저장할 폴더가 없으면 생성
output_folder = 'tortoise_output_seeker'
os.makedirs(output_folder, exist_ok=True)

# Tortoise TTS 모델 초기화
tts = TextToSpeech(use_deepspeed=True, kv_cache=True, half=True)

# 여러 샘플 오디오 파일을 로드하여 화자 샘플로 사용
sample_audio_files = ["storyofthetreasureseekersversion3_01_nesbit_64kb.mp3", "storyofthetreasureseekersversion3_02_nesbit_64kb.mp3", "storyofthetreasureseekersversion3_03_nesbit_64kb.mp3"]
voice_samples = [utils.audio.load_audio(sample, 22050) for sample in sample_audio_files]

In [13]:
# Tortoise에 여러 샘플로 화자 프로필 생성
conditioning_latents = tts.get_conditioning_latents(voice_samples)

# 알파벳을 원하는 발음으로 대체하는 전처리 함수
def replace_alphabets(text):
    alphabet_dict = {
        'A': '에이', 'B': '비', 'C': '씨', 'D': '디', 'E': '이',
        'F': '에프', 'G': '지', 'H': '에이치', 'I': '아이', 'J': '제이',
        'K': '케이', 'L': '엘', 'M': '엠', 'N': '엔', 'O': '오',
        'P': '피', 'Q': '큐', 'R': '아르', 'S': '에스', 'T': '티',
        'U': '유', 'V': '브이', 'W': '더블유', 'X': '엑스', 'Y': '와이', 'Z': '지'
    }
    words = text.split()
    return ' '.join([alphabet_dict.get(word, word) for word in words])

# 텍스트 파일 경로
input_file = "input_level1.txt"  # 텍스트 파일 경로

c:\Users\user\Desktop\빅데이터9기_수업자료_이수지\2024.11.04_(중간프로젝트)\tortoise-tts\tortoise\models\arch_util.py:314: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.mel_norms = torch

In [14]:
# 텍스트 파일을 한 줄씩 읽어서 음성 변환 및 저장
with open(input_file, "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        # 줄바꿈 제거
        text = line.strip()
        
        # 빈 줄은 건너뛰기
        if not text:
            continue

        # 텍스트를 화자 샘플의 스타일로 음성 변환
        pcm_audio = tts.tts_with_preset(text, conditioning_latents=conditioning_latents, preset="ultra_fast")

        # PCM 데이터를 numpy 배열로 변환 후 최종 WAV 파일로 저장
        output_path = os.path.join(output_folder, f'tortoise_ultra_fast_{idx+1}.wav')
        write(output_path, 22050, np.array(pcm_audio, dtype=np.float32))

        print(f"'{text}' 문장이 '{output_path}'로 저장되었습니다.")


c:\Users\user\anaconda3\Lib\site-packages\torch\amp\autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


Generating autoregressive samples..


100%|██████████| 16/16 [00:34<00:00,  2.15s/it]


Computing best candidates using CLVP


100%|██████████| 16/16 [00:07<00:00,  2.20it/s]


...

Transforming autoregressive outputs into audio..


100%|██████████| 30/30 [00:05<00:00,  5.71it/s]


'A, is for alligator.' 문장이 'tortoise_output_seeker\tortoise_ultra_fast_3.wav'로 저장되었습니다.
Generating autoregressive samples..


  0%|          | 0/16 [00:01<?, ?it/s]


KeyboardInterrupt: 

### mp3 -> wav -> embedding

In [1]:
import os
import numpy as np
import torch
from tortoise.api import TextToSpeech
from pydub import AudioSegment
from scipy.io.wavfile import read as read_wav

# Tortoise TTS 초기화
tts = TextToSpeech(use_deepspeed=True, kv_cache=True, half=True)

# MP3 파일이 들어있는 폴더 경로와 변환된 WAV 파일을 저장할 폴더 경로
mp3_folder = "emilyclimbs_2204_librivox"  # MP3 파일 폴더 경로
wav_folder = "emilyclimbs_wav_files"  # WAV 파일이 저장될 폴더 경로
os.makedirs(wav_folder, exist_ok=True)

# 변환된 WAV 파일을 저장할 리스트
wav_files = []

# MP3 파일을 WAV로 변환
for mp3_file in os.listdir(mp3_folder):
    if mp3_file.endswith(".mp3"):
        mp3_path = os.path.join(mp3_folder, mp3_file)
        
        # MP3를 로드하여 WAV로 변환
        audio = AudioSegment.from_mp3(mp3_path)
        audio = audio.set_frame_rate(22050).set_channels(1)  # 샘플링 레이트와 채널 설정
        wav_path = os.path.join(wav_folder, mp3_file.replace(".mp3", ".wav"))
        audio.export(wav_path, format="wav")
        wav_files.append(wav_path)
        print(f"'{mp3_file}'이 '{wav_path}'로 변환되었습니다.")

# 변환된 WAV 파일들을 로드하여 임베딩 생성
voice_samples = []
for wav_file in wav_files:
    try:
        # WAV 파일을 읽고 정규화하여 텐서로 변환
        sr, wav_audio = read_wav(wav_file)
        
        # 스테레오인 경우 모노로 변환
        if wav_audio.ndim > 1:
            wav_audio = wav_audio.mean(axis=1)
        
        # 정규화: -1 ~ 1 범위로 스케일 조정
        wav_audio = wav_audio / np.max(np.abs(wav_audio))  # 오디오 데이터를 -1에서 1 사이로 스케일 조정
        
        # torch.Tensor로 변환하고 차원 추가
        audio_tensor = torch.tensor(wav_audio, dtype=torch.float32).unsqueeze(0)  # (1, 샘플 길이)
        voice_samples.append(audio_tensor)
        print(f"WAV 파일 '{wav_file}'이 성공적으로 로드되어 텐서로 변환되었습니다.")
    except Exception as e:
        print(f"{wav_file} 로드 중 오류 발생: {e}")

# 임베딩 생성
conditioning_latents = tts.get_conditioning_latents(voice_samples)

# 임베딩을 저장할 경로
embedding_path = "saved_embeddings/conditioning_latents_emilyclimbs.npz"
os.makedirs(os.path.dirname(embedding_path), exist_ok=True)
np.savez(embedding_path, *conditioning_latents)
print(f"화자 임베딩이 '{embedding_path}'에 저장되었습니다.")


c:\Users\user\Desktop\빅데이터9기_수업자료_이수지\2024.11.04_(중간프로젝트)\tortoise-tts\tortoise\api.py:221: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.autoregressive.load_state_dict

'emilyclimbs_01_montgomery_64kb.mp3'이 'emilyclimbs_wav_files\emilyclimbs_01_montgomery_64kb.wav'로 변환되었습니다.
'emilyclimbs_02_montgomery_64kb.mp3'이 'emilyclimbs_wav_files\emilyclimbs_02_montgomery_64kb.wav'로 변환되었습니다.
'emilyclimbs_03_montgomery_64kb.mp3'이 'emilyclimbs_wav_files\emilyclimbs_03_montgomery_64kb.wav'로 변환되었습니다.
'emilyclimbs_04_montgomery_64kb.mp3'이 'emilyclimbs_wav_files\emilyclimbs_04_montgomery_64kb.wav'로 변환되었습니다.
'emilyclimbs_05_montgomery_64kb.mp3'이 'emilyclimbs_wav_files\emilyclimbs_05_montgomery_64kb.wav'로 변환되었습니다.
'emilyclimbs_06_Montgomery_64kb.mp3'이 'emilyclimbs_wav_files\emilyclimbs_06_Montgomery_64kb.wav'로 변환되었습니다.
'emilyclimbs_07_montgomery_64kb.mp3'이 'emilyclimbs_wav_files\emilyclimbs_07_montgomery_64kb.wav'로 변환되었습니다.
'emilyclimbs_08_montgomery_64kb.mp3'이 'emilyclimbs_wav_files\emilyclimbs_08_montgomery_64kb.wav'로 변환되었습니다.
'emilyclimbs_09_montgomery_64kb.mp3'이 'emilyclimbs_wav_files\emilyclimbs_09_montgomery_64kb.wav'로 변환되었습니다.
'emilyclimbs_10_montgomery_64kb.mp3'이

c:\Users\user\Desktop\빅데이터9기_수업자료_이수지\2024.11.04_(중간프로젝트)\tortoise-tts\tortoise\models\arch_util.py:314: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.mel_norms = torch

화자 임베딩이 'saved_embeddings/conditioning_latents_emilyclimbs.npz'에 저장되었습니다.


### m4b -> wav 변환

In [6]:
!ffmpeg -i GalacticPatrol2_librivox.m4b -ac 1 -ar 22050 GalacticPatrol2.wav

ffmpeg version 7.0.2-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-frei0r --enable-libfreetype --enable-libfribidi --enable-libharfbuzz --enable-li

### 임베딩 저장

In [1]:
import os
import numpy as np
import torch
from tortoise.api import TextToSpeech
from scipy.io.wavfile import read as read_wav

In [2]:
# Tortoise TTS 모델 초기화
tts = TextToSpeech(use_deepspeed=True, kv_cache=True, half=True)

wav_path = "GalacticPatrol2.wav"  # 변환된 WAV 파일 경로

# WAV 파일을 로드하고 텐서로 변환
try:
    sr, wav_audio = read_wav(wav_path)
    
    # 스테레오인 경우 모노로 변환
    if wav_audio.ndim > 1:
        wav_audio = wav_audio.mean(axis=1)

    # 정규화: -1 ~ 1 범위로 스케일 조정
    wav_audio = wav_audio / np.max(np.abs(wav_audio))  # 오디오 데이터를 -1에서 1 사이로 스케일 조정

    # torch.Tensor로 변환하고 차원 추가
    audio_tensor = torch.tensor(wav_audio, dtype=torch.float32).unsqueeze(0)
    print(f"WAV 파일이 성공적으로 로드되어 텐서로 변환되었습니다.")
except Exception as e:
    print(f"WAV 파일을 로드하는 중 오류 발생: {e}")
    raise



c:\Users\user\Desktop\빅데이터9기_수업자료_이수지\2024.11.04_(중간프로젝트)\tortoise-tts\tortoise\api.py:221: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.autoregressive.load_state_dict

WAV 파일이 성공적으로 로드되어 텐서로 변환되었습니다.


In [3]:
# Tortoise TTS에서 화자 프로필 임베딩 생성
conditioning_latents = tts.get_conditioning_latents([audio_tensor])

# 임베딩 저장
embedding_path = "saved_embeddings/conditioning_latents_GalacticPatrol2"
os.makedirs(os.path.dirname(embedding_path), exist_ok=True)
np.savez(embedding_path, *conditioning_latents)
print(f"화자 임베딩이 '{embedding_path}'에 저장되었습니다.")

c:\Users\user\Desktop\빅데이터9기_수업자료_이수지\2024.11.04_(중간프로젝트)\tortoise-tts\tortoise\models\arch_util.py:314: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.mel_norms = torch

화자 임베딩이 'saved_embeddings/conditioning_latents_GalacticPatrol2'에 저장되었습니다.


### BeconSecondReader 임베딩으로 음성 추출

In [7]:
import os
import numpy as np
import torch
from tortoise.api import TextToSpeech
from scipy.io.wavfile import write

# 저장된 임베딩 로드 및 Tensor로 변환
embedding_path = "saved_embeddings1/conditioning_latents.npz"
loaded_latents = np.load(embedding_path)
conditioning_latents = [torch.tensor(loaded_latents[key]) for key in loaded_latents]

# Tortoise TTS 모델 초기화
tts = TextToSpeech(use_deepspeed=True, kv_cache=True, half=True)

# 텍스트 파일 경로
input_file = "input_level1.txt"  # 변환할 텍스트 파일 경로
output_folder = "tortoise_output_standard_level1_embeddings1"
os.makedirs(output_folder, exist_ok=True)

# 텍스트 파일을 한 줄씩 읽어서 음성 변환 및 저장
with open(input_file, "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        # 줄바꿈 제거 후 전처리 적용
        text = line.strip()
        if not text:
            continue

        # 텍스트를 화자 샘플의 스타일로 음성 변환
        pcm_audio = tts.tts_with_preset(text, conditioning_latents=conditioning_latents, preset="ultra_fast")

        # PCM 데이터를 numpy 배열로 변환 후 최종 WAV 파일로 저장
        output_path = os.path.join(output_folder, f'tortoise_test_ultra_fast_{idx+1}.wav')
        write(output_path, 22050, np.array(pcm_audio, dtype=np.float32))

        print(f"'{text}' 문장이 '{output_path}'로 저장되었습니다.")

Generating autoregressive samples..


100%|██████████| 16/16 [00:33<00:00,  2.11s/it]


Computing best candidates using CLVP


100%|██████████| 16/16 [00:09<00:00,  1.65it/s]


Transforming autoregressive outputs into audio..


...

Transforming autoregressive outputs into audio..


100%|██████████| 30/30 [00:06<00:00,  4.52it/s]


'A, is for apple.' 문장이 'tortoise_output_standard_level1_embeddings1\tortoise_test_ultra_fast_2.wav'로 저장되었습니다.
Generating autoregressive samples..


 31%|███▏      | 5/16 [00:12<00:27,  2.49s/it]


KeyboardInterrupt: 

### 피노키오 음성으로 추출

In [ ]:
## 피노키오

import os
import numpy as np
import torch
from tortoise.api import TextToSpeech
from scipy.io.wavfile import write

# 저장된 임베딩 로드 및 Tensor로 변환
embedding_path = "saved_embeddings/conditioning_latents_pino.npz"
loaded_latents = np.load(embedding_path)
conditioning_latents = [torch.tensor(loaded_latents[key]) for key in loaded_latents]

# Tortoise TTS 모델 초기화
tts = TextToSpeech(use_deepspeed=True, kv_cache=True, half=True)

# 텍스트 파일 경로
input_file = "input_level1.txt"  # 변환할 텍스트 파일 경로
output_folder = "tortoise_output_standard_level1"
os.makedirs(output_folder, exist_ok=True)

# 텍스트 파일을 한 줄씩 읽어서 음성 변환 및 저장
with open(input_file, "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        # 줄바꿈 제거 후 전처리 적용
        text = line.strip()
        if not text:
            continue

        # 텍스트를 화자 샘플의 스타일로 음성 변환
        pcm_audio = tts.tts_with_preset(text, conditioning_latents=conditioning_latents, preset="ultra_fast")

        # PCM 데이터를 numpy 배열로 변환 후 최종 WAV 파일로 저장
        output_path = os.path.join(output_folder, f'tortoise_test_ultra_fast_{idx+1}.wav')
        write(output_path, 22050, np.array(pcm_audio, dtype=np.float32))

        print(f"'{text}' 문장이 '{output_path}'로 저장되었습니다.")


c:\Users\user\Desktop\빅데이터9기_수업자료_이수지\2024.11.04_(중간프로젝트)\tortoise-tts\tortoise\api.py:221: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.autoregressive.load_state_dict

Generating autoregressive samples..


100%|██████████| 16/16 [01:03<00:00,  3.97s/it]


Computing best candidates using CLVP


100%|██████████| 16/16 [00:42<00:00,  2.63s/it]


...

100%|██████████| 30/30 [00:06<00:00,  4.76it/s]


'I, is for idea.' 문장이 'tortoise_output_standard_level1\tortoise_test_ultra_fast_45.wav'로 저장되었습니다.
Generating autoregressive samples..


100%|██████████| 16/16 [00:35<00:00,  2.23s/it]


Computing best candidates using CLVP


 81%|████████▏ | 13/16 [00:07<00:01,  1.67it/s]

### kittys 임베딩으로 음성 추출

In [ ]:
import os
import numpy as np
import torch
from tortoise.api import TextToSpeech
from scipy.io.wavfile import write

# 저장된 임베딩 로드 및 Tensor로 변환
embedding_path = "saved_embeddings/conditioning_latents_KittysEnemy.npz"
loaded_latents = np.load(embedding_path)
conditioning_latents = [torch.tensor(loaded_latents[key]) for key in loaded_latents]

# Tortoise TTS 모델 초기화
tts = TextToSpeech(use_deepspeed=True, kv_cache=True, half=True)

# 텍스트 파일 경로
input_file = "input_level1.txt"  # 변환할 텍스트 파일 경로
output_folder = "tortoise_output_kittys_level1"
os.makedirs(output_folder, exist_ok=True)

# 텍스트 파일을 한 줄씩 읽어서 음성 변환 및 저장
with open(input_file, "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        # 줄바꿈 제거 후 텍스트에 '[enunciate]' 태그 추가
        text = line.strip()
        if not text:
            continue
        processed_text = f"[enunciate] {text}"

        # 텍스트를 화자 샘플의 스타일로 음성 변환
        pcm_audio = tts.tts_with_preset(processed_text, conditioning_latents=conditioning_latents, preset="ultra_fast")

        # PCM 데이터를 numpy 배열로 변환 후 최종 WAV 파일로 저장
        output_path = os.path.join(output_folder, f'tortoise_test_ultra_fast_{idx+1}.wav')
        write(output_path, 22050, np.array(pcm_audio, dtype=np.float32))

        print(f"'{processed_text}' 문장이 '{output_path}'로 저장되었습니다.")


Generating autoregressive samples..


100%|██████████| 16/16 [03:48<00:00, 14.25s/it]


Computing best candidates using CLVP


100%|██████████| 16/16 [00:09<00:00,  1.71it/s]


Transforming autoregressive outputs into audio..


100%|██████████| 30/30 [00:06<00:00,  4.48it/s]


AssertionError: Torch not compiled with CUDA enabled

In [11]:
import tortoise


### threads 설정

In [3]:
import os
import numpy as np
import torch
from tortoise.api import TextToSpeech
from scipy.io.wavfile import write

In [ ]:

import os
import numpy as np
import torch
from tortoise.api import TextToSpeech
from scipy.io.wavfile import write

# CPU에서 사용할 스레드 수 설정
torch.set_num_threads(10)  # 자신의 CPU 코어 수에 맞게 설정하세요.

# 저장된 임베딩 로드 및 Tensor로 변환
embedding_path = "saved_embeddings/conditioning_latents_EightGirl.npz"
loaded_latents = np.load(embedding_path)
conditioning_latents = [torch.tensor(loaded_latents[key]) for key in loaded_latents]

# Tortoise TTS 모델 초기화
tts = TextToSpeech(use_deepspeed=True, kv_cache=True, half=True)

# 텍스트 파일 경로
input_file = "input_level1.txt"  # 변환할 텍스트 파일 경로
output_folder = "tortoise_output_EightGirl_level1"
os.makedirs(output_folder, exist_ok=True)

# 텍스트 파일을 한 줄씩 읽어서 음성 변환 및 저장
with open(input_file, "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        # 줄바꿈 제거 후 전처리 적용
        text = line.strip()
        if not text:
            continue

        # 텍스트를 화자 샘플의 스타일로 음성 변환
        pcm_audio = tts.tts_with_preset(text, conditioning_latents=conditioning_latents, preset="ultra_fast")

        # PCM 데이터를 numpy 배열로 변환 후 최종 WAV 파일로 저장
        output_path = os.path.join(output_folder, f'tortoise_test_ultra_fast_{idx+1}.wav')
        write(output_path, 22050, np.array(pcm_audio, dtype=np.float32))

        print(f"'{text}' 문장이 '{output_path}'로 저장되었습니다.")



Generating autoregressive samples..


100%|██████████| 16/16 [00:56<00:00,  3.56s/it]


Computing best candidates using CLVP


100%|██████████| 16/16 [00:46<00:00,  2.92s/it]


Transforming autoregressive outputs into audio..


...

'F, is for foot.' 문장이 'tortoise_output_EightGirl_level1\tortoise_test_ultra_fast_29.wav'로 저장되었습니다.
Generating autoregressive samples..


100%|██████████| 16/16 [00:35<00:00,  2.23s/it]


Computing best candidates using CLVP


 25%|██▌       | 4/16 [00:02<00:07,  1.58it/s]


KeyboardInterrupt: 

### threads & alphabet 

In [ ]:
import os
import numpy as np
import torch
from tortoise.api import TextToSpeech
from scipy.io.wavfile import write

# CPU에서 사용할 스레드 수 설정
torch.set_num_threads(20)  # 자신의 CPU 코어 수에 맞게 설정하세요.

# 알파벳을 영어식 철자 발음으로 대체하는 전처리 함수
def replace_alphabets(text):
    alphabet_dict = {
        'A': '에이', 'B': 'bee', 'C': 'cee', 'D': 'dee', 'E': 'ee',
        'F': 'ef', 'G': 'gee', 'H': 'aitch', 'I': 'eye', 'J': 'jay',
        'K': 'kay', 'L': 'el', 'M': 'em', 'N': 'en', 'O': 'oh',
        'P': 'pee', 'Q': 'queue', 'R': 'ar', 'S': 'es', 'T': 'tee',
        'U': 'you', 'V': 'vee', 'W': 'double-u', 'X': 'ex', 'Y': 'why', 'Z': 'zee'
    }
    words = text.split()
    return ' '.join([alphabet_dict.get(word.upper(), word) for word in words])

# 저장된 임베딩 로드 및 Tensor로 변환
embedding_path = "saved_embeddings/conditioning_latents_DayBirthdayDol.npz"
loaded_latents = np.load(embedding_path)
conditioning_latents = [torch.tensor(loaded_latents[key]) for key in loaded_latents]

# Tortoise TTS 모델 초기화
tts = TextToSpeech(use_deepspeed=True, kv_cache=True, half=True)

# 텍스트 파일 경로
input_file = "input_level1.txt"  # 변환할 텍스트 파일 경로
output_folder = "tortoise_output_DayBirthdayDol_level1"
os.makedirs(output_folder, exist_ok=True)

# 텍스트 파일을 한 줄씩 읽어서 음성 변환 및 저장
with open(input_file, "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        # 줄바꿈 제거 후 알파벳 발음 전처리 적용
        text = line.strip()
        if not text:
            continue
        processed_text = replace_alphabets(text)

        # 텍스트를 화자 샘플의 스타일로 음성 변환
        pcm_audio = tts.tts_with_preset(processed_text, conditioning_latents=conditioning_latents, preset="fast")

        # PCM 데이터를 numpy 배열로 변환 후 최종 WAV 파일로 저장
        output_path = os.path.join(output_folder, f'DayBirthdayDo_fast_{idx+1}.wav')
        write(output_path, 22050, np.array(pcm_audio, dtype=np.float32))

        print(f"'{processed_text}' 문장이 '{output_path}'로 저장되었습니다.")


Generating autoregressive samples..


100%|██████████| 96/96 [03:13<00:00,  2.01s/it]


Computing best candidates using CLVP


100%|██████████| 96/96 [00:44<00:00,  2.15it/s]


Transforming autoregressive outputs into audio..


...

100%|██████████| 96/96 [01:21<00:00,  1.18it/s]


Transforming autoregressive outputs into audio..


100%|██████████| 80/80 [00:22<00:00,  3.61it/s]


'em is for milk' 문장이 'tortoise_output_DayBirthdayDol_level1\DayBirthdayDo_fast_63.wav'로 저장되었습니다.
Generating autoregressive samples..


 11%|█▏        | 11/96 [00:20<02:39,  1.88s/it]


### 후처리

In [ ]:
## 후처리1

import os
import librosa
import noisereduce as nr
import soundfile as sf

# 입력 폴더 및 출력 폴더 설정
input_folder = "bark_output_9_enunciate"
output_folder = "bark_output_9_slow_denoised"
os.makedirs(output_folder, exist_ok=True)  # 출력 폴더가 없으면 생성

# 입력 폴더에 있는 모든 .wav 파일을 순서대로 처리
for filename in os.listdir(input_folder):
    if filename.endswith(".wav"):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)
        
        # 오디오 파일 로드
        audio, sr = librosa.load(input_path, sr=None)
        
        # 잡음 제거 적용
        reduced_noise_audio = nr.reduce_noise(y=audio, sr=sr)
        
        # 잡음 제거된 오디오 파일 저장
        sf.write(output_path, reduced_noise_audio, sr, format='WAV')
        
        print(f"'{filename}' 파일이 잡음 제거되어 '{output_path}'에 저장되었습니다.")


In [ ]:
## 후처리2

import librosa
import noisereduce as nr
import soundfile as sf

# 오디오 파일 로드
input_path = "/mnt/data/DayBirthdayDo_fast_1.wav"
output_path = "/mnt/data/DayBirthdayDo_fast_1_processed.wav"
y, sr = librosa.load(input_path, sr=22050)

# 1. 잡음 제거
reduced_noise = nr.reduce_noise(y=y, sr=sr)

# 2. 피치 조정 (예: n_steps=-2는 피치를 약간 낮추는 설정)
y_pitch_shifted = librosa.effects.pitch_shift(reduced_noise, sr, n_steps=-2)

# 3. 속도 조정 (예: 1.1배 빠르게)
y_time_stretched = librosa.effects.time_stretch(y_pitch_shifted, rate=1.1)

# 4. 파일 저장
sf.write(output_path, y_time_stretched, sr)
print(f"후처리된 오디오 파일이 '{output_path}'에 저장되었습니다.")


### level2 / ultra_fast / embedding 

In [19]:
import os
import numpy as np
import torch
from tortoise.api import TextToSpeech
from scipy.io.wavfile import write

In [20]:
# CPU에서 사용할 스레드 수 설정
torch.set_num_threads(20)  # 자신의 CPU 코어 수에 맞게 설정하세요.

# 저장된 임베딩 로드 및 Tensor로 변환
embedding_path = "saved_embeddings/conditioning_latents_SelectedWorksOfE.npz"
loaded_latents = np.load(embedding_path)
conditioning_latents = [torch.tensor(loaded_latents[key], device='cpu') for key in loaded_latents]

# Tortoise TTS 모델 초기화 (CPU 전용 설정)
tts = TextToSpeech(use_deepspeed=False, kv_cache=False, half=False)

# 텍스트 파일 경로
input_file = "input_level2.txt"  # 변환할 텍스트 파일 경로
output_folder = "tortoise_output_SelectedWorksOfE_level2"
os.makedirs(output_folder, exist_ok=True)

# 텍스트 파일을 한 줄씩 읽어서 음성 변환 및 저장
with open(input_file, "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        # 줄바꿈 제거 및 '[enunciate]' 태그 추가
        text = line.strip()
        if not text:
            continue
        processed_text = f"[enunciate] {text}"

        # 텍스트를 화자 샘플의 스타일로 음성 변환
        pcm_audio = tts.tts_with_preset(processed_text, conditioning_latents=conditioning_latents, preset="ultra_fast")

        # PCM 데이터를 numpy 배열로 변환 후 최종 WAV 파일로 저장
        output_path = os.path.join(output_folder, f'SelectedWorksOfE_ultra_fast_{idx+1}.wav')
        write(output_path, 22050, np.array(pcm_audio, dtype=np.float32))

        print(f"'{processed_text}' 문장이 '{output_path}'로 저장되었습니다.")

Generating autoregressive samples..


100%|██████████| 16/16 [01:09<00:00,  4.36s/it]


Computing best candidates using CLVP


100%|██████████| 16/16 [00:08<00:00,  1.95it/s]


Transforming autoregressive outputs into audio..


100%|██████████| 30/30 [00:05<00:00,  5.49it/s]


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
import os
import numpy as np
import torch
from tortoise.api import TextToSpeech
from scipy.io.wavfile import write

In [2]:
import os
import numpy as np
import torch
from tortoise.api import TextToSpeech
from scipy.io.wavfile import write
from pydub import AudioSegment

# 오디오 파일을 확실하게 WAV 형식으로 변환
embedding_path = "librivox_m4b_files/AlicesAdventuresUnderground.wav"
audio = AudioSegment.from_file(embedding_path)
audio = audio.set_frame_rate(22050).set_channels(1)  # 샘플링 레이트와 모노로 설정
audio_data = np.array(audio.get_array_of_samples(), dtype=np.float32)

conditioning_latents = [torch.tensor(audio_data, dtype=torch.float32)]

# Tortoise TTS 모델 초기화
tts = TextToSpeech(use_deepspeed=True, kv_cache=True, half=True)

# 텍스트 파일 경로
input_file = "input_level1.txt"  # 변환할 텍스트 파일 경로
output_folder = "tortoise_output_AlicesAdventuresUnderground_level1"
os.makedirs(output_folder, exist_ok=True)

# 텍스트 파일을 한 줄씩 읽어서 음성 변환 및 저장
with open(input_file, "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        # 줄바꿈 제거 후 전처리 적용
        text = line.strip()
        if not text:
            continue

        # 텍스트를 화자 샘플의 스타일로 음성 변환
        pcm_audio = tts.tts_with_preset(text, conditioning_latents=conditioning_latents, preset="ultra_fast")

        # PCM 데이터를 numpy 배열로 변환 후 최종 WAV 파일로 저장
        output_path = os.path.join(output_folder, f'tortoise_test_ultra_fast_{idx+1}.wav')
        write(output_path, 22050, np.array(pcm_audio, dtype=np.float32))

        print(f"'{text}' 문장이 '{output_path}'로 저장되었습니다.")


ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
import os
import numpy as np
from tortoise.api import TextToSpeech
from tortoise import utils
from scipy.io.wavfile import write

In [ ]:
# 저장할 폴더가 없으면 생성
output_folder = 'tortoise_output_standard_level1'
os.makedirs(output_folder, exist_ok=True)

# Tortoise TTS 모델 초기화
tts = TextToSpeech(use_deepspeed=True, kv_cache=True, half=True)

# 여러 샘플 오디오 파일을 로드하여 화자 샘플로 사용
sample_audio_files = []
voice_samples = [utils.audio.load_audio(sample, 22050) for sample in sample_audio_files]

In [ ]:
# Tortoise에 여러 샘플로 화자 프로필 생성
conditioning_latents = tts.get_conditioning_latents(voice_samples)

# 알파벳을 영어식 철자 발음으로 대체하는 전처리 함수
def replace_alphabets(text):
    alphabet_dict = {
        'A': 'ay', 'B': 'bee', 'C': 'cee', 'D': 'dee', 'E': 'ee',
        'F': 'ef', 'G': 'gee', 'H': 'aitch', 'I': 'eye', 'J': 'jay',
        'K': 'kay', 'L': 'el', 'M': 'em', 'N': 'en', 'O': 'oh',
        'P': 'pee', 'Q': 'queue', 'R': 'ar', 'S': 'es', 'T': 'tee',
        'U': 'you', 'V': 'vee', 'W': 'double-u', 'X': 'ex', 'Y': 'why', 'Z': 'zee'
    }
    words = text.split()
    return ' '.join([alphabet_dict.get(word, word) for word in words])

# 텍스트 파일 경로
input_file = "input_level1.txt"  # 텍스트 파일 경로

In [ ]:
# 텍스트 파일을 한 줄씩 읽어서 음성 변환 및 저장
with open(input_file, "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        # 줄바꿈 제거
        text = line.strip()
        
        # 빈 줄은 건너뛰기
        if not text:
            continue

        # 텍스트를 화자 샘플의 스타일로 음성 변환
        pcm_audio = tts.tts_with_preset(text, conditioning_latents=conditioning_latents, preset="standard")

        # PCM 데이터를 numpy 배열로 변환 후 최종 WAV 파일로 저장
        output_path = os.path.join(output_folder, f'tortoise_test_ultra_fast_{idx+1}.wav')
        write(output_path, 22050, np.array(pcm_audio, dtype=np.float32))

        print(f"'{text}' 문장이 '{output_path}'로 저장되었습니다.")
